In [415]:
import pyautogui as gui
import cv2
import time
import pytesseract
import matplotlib.pyplot as plt
import numpy as np

In [436]:
class Airplane():
    def __init__(self, flightnum = None, model = None, current_height = 0, target_height = 0, speed = 0, position = (0,0), heading = "SPL"):
        self.flightnum = flightnum
        self.info = ""
        self.model = model
        self.current_height = current_height
        self.target_height = target_height
        self.speed = speed
        self.heading = heading
        self.position = position
        
    def __repr__(self):
        return "<Airplane flightnum:%s position:%s>" % (self.flightnum, self.position)
        
    def __str__(self):
        return "%s%s, screen position: %s, passing flight level %d to flight level %d, speed %d knots, %s." % (self.flightnum, self.model, str(self.position), self.current_height, self.target_height, self.speed, self.heading)

In [417]:
def ocr(img):
    pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
    text = pytesseract.image_to_string(img, lang='eng')
    return text

In [418]:
def get_plane_info(img):
    top_idx = 29
    bot_idx = 1029
    screen_idx = 368
    info_idx = 977
    panel = img[top_idx:bot_idx, :screen_idx, :]
    screen = img[top_idx:bot_idx, screen_idx:, :]
    line = screen[info_idx-15:info_idx+5]
    text = ocr(line).split('.')[0]
    return text

In [419]:
def remove_same(pts, threshold):
    elements = []
    for x,y in pts:
        for ele in elements:
            if ((x-ele[0])**2 + (y-ele[1])**2) < threshold**2:
                break
        else:
            elements.append((x,y))
    
    return elements

In [420]:
def find_plane_pts(img, template, threshold=0.9, method=cv2.TM_CCORR_NORMED):
    # Apply template Matching
    resultimg = img.copy()
    res = cv2.matchTemplate(resultimg, template, method)

    location = np.where(res >= threshold)
#     print(location)
    pts = zip(*location[::-1])
    plane_pts = remove_same(pts, min(template.shape[0], template.shape[1]))
    
    color = (0, 0, 255)
    for pt in plane_pts:
        cv2.rectangle(resultimg, (pt[0], pt[1]), (pt[0]+17, pt[1]+17), color, 2)
    
    plt.imsave("result.png", resultimg)
    return plane_pts

In [437]:
time.sleep(5)
current_img = cv2.cvtColor(np.array(gui.screenshot()), cv2.COLOR_RGB2BGR)
bonding_box = cv2.imread('2ktempl.png')

plane_pts = find_plane_pts(current_img, bonding_box)
print(plane_pts)

[(1446, 355), (845, 401), (1213, 440), (1092, 503), (1035, 579), (772, 787), (898, 916)]


In [438]:
airplanes = []
for pt in plane_pts:
    print(pt)
    gui.click(x=pt[0], y=pt[1])
    info_img = cv2.cvtColor(np.array(gui.screenshot()), cv2.COLOR_RGB2BGR)
    info = get_plane_info(info_img)
    airplane = Airplane(pt)
    airplane.info = info
    airplanes.append(airplane)
    print(info)
    
    
print(airplanes)

(1446, 355)
DLH380H (A319), maintaining flight level 70, speed 250 knots, direct to SPL
(845, 401)
MNB1967 (A332), maintaining flight level 70, speed 250 knots, direct to SPL
(1213, 440)
EJU32E (A319), maintaining flight level 60, speed 250, direct to NYKER
(1092, 503)
CWC75W (B744) is taking off, cleared to flight level 60, speed 158
(1035, 579)
KLM85X (B789), maintaining flight level 70, speed 220 knots, direct to SPL
(772, 787)
KLM248G (B737), maintaining flight level 70, speed 250 knots, direct to SPL
(898, 916)
KLM36Y (B77W), maintaining flight level 70, speed 250 knots, direct to SPL
[<Airplane flightnum:(1446, 355) position:(0, 0)>, <Airplane flightnum:(845, 401) position:(0, 0)>, <Airplane flightnum:(1213, 440) position:(0, 0)>, <Airplane flightnum:(1092, 503) position:(0, 0)>, <Airplane flightnum:(1035, 579) position:(0, 0)>, <Airplane flightnum:(772, 787) position:(0, 0)>, <Airplane flightnum:(898, 916) position:(0, 0)>]


In [442]:
print(airplanes[2].info)

EJU32E (A319), maintaining flight level 60, speed 250, direct to NYKER


In [424]:
# # Calibration
# choose_img = cv2.imread("test.png")
# print(choose_img.shape)
# gray = cv2.cvtColor(choose_img, cv2.COLOR_BGR2GRAY)
# for i in range(gray.shape[1]):
#     if gray[300, i] == 11:
#         screen_panel_idx = i
#         print(i)
#         break
        
# for i in range(gray.shape[0]):
#     if gray[i, 800] == 11:
#         top_idx = i
#         print(i)
#         break
        
# for i in reversed(range(gray.shape[0])):
#     if gray[i, 800] == 11:
#         bot_idx = i
#         print(i)
#         break

# panel = choose_img[top_idx:bot_idx, :screen_idx, :]
# screen = choose_img[top_idx:bot_idx, screen_idx:, :]
# print(panel.shape, screen.shape)
# plt.imshow(panel)
# plt.imshow(screen)
# plt.imsave("screen.png", screen)

# grayscreen = cv2.cvtColor(screen, cv2.COLOR_BGR2GRAY)
# # plt.imshow(grayscreen, cmap='gray', vmin=0, vmax=255)
# # plt.imsave("grayscreen.png", grayscreen, cmap='gray')
# # blacknum = np.zeros(screen.shape[0])
# threshold = 1400
# for i in reversed(range(grayscreen.shape[0])):
#     count = np.count_nonzero(grayscreen[i] == 11)
# #     blacknum[i] = count
#     if count < threshold:
#         info_idx = i
#         print(i)
#         break;

In [425]:
# infoimg = screen[info_idx-15:info_idx+5]
# # plt.imshow(infoimg)
# text = ocr(infoimg)
# print(text)

# info = text.split(',')
# flightnum = info[0].split()[0]
# model = info[0].split()[1]
# current_height = int(info[1].split()[3])
# target_height = int(info[1].split()[8])
# speed = int(info[2].split()[1])
# heading = info[3]
# plane_find = Airplane(flightnum, model, current_height, target_height, speed, heading=heading)
# print(plane_find)

In [426]:
#     info = text.split(',')
#     flightnum = info[0].split()[0]
#     model = info[0].split()[1]
#     current_height = int(info[1].split()[3])
#     target_height = int(info[1].split()[8])
#     speed = int(info[2].split()[1])
#     heading = info[3]
#     print(plane_find)